Part 3: Writing your own Elman RNN

In [1]:
%run Loading_data.ipynb

In [2]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

In [7]:
batch = []
target = []
for j in range(500):
    tensor_x = []
    for i in range(40):
        tensor_x.append(torch.tensor(x_train[i+j*40], dtype=torch.long))
    tensor_pad = pad_sequence(tensor_x, batch_first=True, padding_value=0)
    batch.append(tensor_pad)
    print(tensor_pad.shape)
    target.append(torch.tensor(y_train[0+j*40:40+j*40]))

torch.Size([40, 32])
torch.Size([40, 37])
torch.Size([40, 40])
torch.Size([40, 42])
torch.Size([40, 44])
torch.Size([40, 45])
torch.Size([40, 46])
torch.Size([40, 47])
torch.Size([40, 49])
torch.Size([40, 50])
torch.Size([40, 51])
torch.Size([40, 52])
torch.Size([40, 53])
torch.Size([40, 54])
torch.Size([40, 55])
torch.Size([40, 56])
torch.Size([40, 57])
torch.Size([40, 59])
torch.Size([40, 59])
torch.Size([40, 60])
torch.Size([40, 62])
torch.Size([40, 63])
torch.Size([40, 64])
torch.Size([40, 65])
torch.Size([40, 66])
torch.Size([40, 67])
torch.Size([40, 69])
torch.Size([40, 70])
torch.Size([40, 71])
torch.Size([40, 72])
torch.Size([40, 73])
torch.Size([40, 74])
torch.Size([40, 75])
torch.Size([40, 76])
torch.Size([40, 77])
torch.Size([40, 78])
torch.Size([40, 79])
torch.Size([40, 80])
torch.Size([40, 81])
torch.Size([40, 82])
torch.Size([40, 84])
torch.Size([40, 85])
torch.Size([40, 86])
torch.Size([40, 87])
torch.Size([40, 88])
torch.Size([40, 89])
torch.Size([40, 90])
torch.Size([4

torch.Size([40, 315])
torch.Size([40, 317])
torch.Size([40, 319])
torch.Size([40, 321])
torch.Size([40, 322])
torch.Size([40, 324])
torch.Size([40, 326])
torch.Size([40, 328])
torch.Size([40, 329])
torch.Size([40, 331])
torch.Size([40, 333])
torch.Size([40, 335])
torch.Size([40, 338])
torch.Size([40, 339])
torch.Size([40, 341])
torch.Size([40, 343])
torch.Size([40, 346])
torch.Size([40, 347])
torch.Size([40, 349])
torch.Size([40, 352])
torch.Size([40, 354])
torch.Size([40, 356])
torch.Size([40, 358])
torch.Size([40, 360])
torch.Size([40, 362])
torch.Size([40, 365])
torch.Size([40, 367])
torch.Size([40, 370])
torch.Size([40, 372])
torch.Size([40, 374])
torch.Size([40, 377])
torch.Size([40, 378])
torch.Size([40, 381])
torch.Size([40, 383])
torch.Size([40, 385])
torch.Size([40, 387])
torch.Size([40, 391])
torch.Size([40, 395])
torch.Size([40, 398])
torch.Size([40, 400])
torch.Size([40, 404])
torch.Size([40, 407])
torch.Size([40, 409])
torch.Size([40, 413])
torch.Size([40, 416])
torch.Size

In [6]:
v_batch = []
v_target = []
for j in range(125):
    tensor_x = []
    for i in range(40):
        tensor_x.append(torch.tensor(x_train[i+j*40], dtype=torch.long))
    tensor_pad = pad_sequence(tensor_x, batch_first=True, padding_value=0)
    v_batch.append(tensor_pad)
    v_target.append(torch.tensor(y_train[0+j*40:40+j*40]))

torch.Size([40, 32])
torch.Size([40, 37])
torch.Size([40, 40])
torch.Size([40, 42])
torch.Size([40, 44])
torch.Size([40, 45])
torch.Size([40, 46])
torch.Size([40, 47])
torch.Size([40, 49])
torch.Size([40, 50])
torch.Size([40, 51])
torch.Size([40, 52])
torch.Size([40, 53])
torch.Size([40, 54])
torch.Size([40, 55])
torch.Size([40, 56])
torch.Size([40, 57])
torch.Size([40, 59])
torch.Size([40, 59])
torch.Size([40, 60])
torch.Size([40, 62])
torch.Size([40, 63])
torch.Size([40, 64])
torch.Size([40, 65])
torch.Size([40, 66])
torch.Size([40, 67])
torch.Size([40, 69])
torch.Size([40, 70])
torch.Size([40, 71])
torch.Size([40, 72])
torch.Size([40, 73])
torch.Size([40, 74])
torch.Size([40, 75])
torch.Size([40, 76])
torch.Size([40, 77])
torch.Size([40, 78])
torch.Size([40, 79])
torch.Size([40, 80])
torch.Size([40, 81])
torch.Size([40, 82])
torch.Size([40, 84])
torch.Size([40, 85])
torch.Size([40, 86])
torch.Size([40, 87])
torch.Size([40, 88])
torch.Size([40, 89])
torch.Size([40, 90])
torch.Size([4

In [7]:
class Elman (nn.Module):
    def __init__ (self , insize= 300 , outsize= 300 , hsize= 300 ):
        super (). __init__ ()
        self.lin1 = nn.Linear(insize+hsize,hsize)  
        self.lin2 = nn.Linear(hsize,outsize)
    def forward (self , x, hidden= None):
        b, t, e = x.size()
        if hidden is None :
            hidden = torch.zeros(b, e, dtype =torch.float)
        outs = []
        
        for i in range (t):
            inp = torch.cat([x[:, i, :], hidden], dim = 1)
            inp = F.relu(self.lin1(inp)) 
            out = self.lin2(inp)
            outs.append(out[:, None , :])
        return torch.cat(outs, dim = 1 ), hidden

In [8]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 16 output channels, 3x3 square convolution
        # kernel
        self.layer1 = nn.Embedding(len(i2w),300)
        # an affine operation: y = Wx + b
        self.fc1 = Elman(300,300,300)  
        self.fc2 = nn.Linear(300,2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.fc1(x)
        x = torch.max(F.relu(x[0]),1)
        x = self.fc2(x.values)
        return x

net_Rnn = Net()
print(net_Rnn)

Net(
  (layer1): Embedding(99430, 300)
  (fc1): Elman(
    (lin1): Linear(in_features=600, out_features=300, bias=True)
    (lin2): Linear(in_features=300, out_features=300, bias=True)
  )
  (fc2): Linear(in_features=300, out_features=2, bias=True)
)


In [9]:
import torch.optim as optim
optimizer = optim.Adam(net_Rnn.parameters(),betas=(0.9, 0.999), lr=0.001)

In [11]:
for epoch in range(1): 
    for j in range(500):
        outputs = net_Rnn(batch[j])
        optimizer.zero_grad()
        loss = torch.nn.functional.cross_entropy(F.softmax(outputs,1),target[j])
        loss.backward()
        optimizer.step()
        print(loss.item())

print('Finished Training')

0.6689584255218506
0.6768030524253845
0.6570640206336975
0.655188798904419
Finished Training


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for j in range(125):
        outputs = net(v_batch[j])
        _, predicted = torch.max(outputs.data, 1)
        total += v_target[j].size(0)
        correct += (predicted == v_target[j]).sum().item()

print('Accuracy of the network on the 5000 test images: %d %%' % (
    100 * correct / total))